In [431]:
url_default = "https://search.naver.com/search.naver"

In [305]:
url_default

'https://search.naver.com/search.naver'

In [469]:
import requests as req, json
from bs4 import BeautifulSoup as bs
from parsel import Selector
from urllib.parse import urljoin
import pandas as pd
from pprint import pprint
from datetime import date 

In [432]:
headers_default = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"}

In [632]:
params_default = {'where': 'news',
'query': '',
'sm': 'tab_opt',
'sort': '0',
'photo': '0',
'field': '0',
'pd': '0',
'ds': "",
'de': "",
'docid':"" ,
'related': '0',
'mynews': '0',
'office_type': '0',
'office_section_code': '0',
'news_office_checked': "",
'nso': 'so:r,p:all',
'is_sug_officeid': '0'}

In [633]:
params_default

{'where': 'news',
 'query': '',
 'sm': 'tab_opt',
 'sort': '0',
 'photo': '0',
 'field': '0',
 'pd': '0',
 'ds': '',
 'de': '',
 'docid': '',
 'related': '0',
 'mynews': '0',
 'office_type': '0',
 'office_section_code': '0',
 'news_office_checked': '',
 'nso': 'so:r,p:all',
 'is_sug_officeid': '0'}

In [467]:
res_default = req.get(url_default, headers=headers_default, params= params_default)
soup = bs(res_default.text, 'html.parser')
option = soup.find('ul', {'class': 'lst_option'})
#pd
time_list  = {'전체': '0','1시간': '7' ,'2시간': '8','3시간': '9' ,'4시간': '10' ,'5시간': '11' ,'6시간': '12' ,'1일': '4', '1주': '1' , '1개월':'2', '3개월': '13','6개월': '6', '1년': '5', 
           '직접입력': '3'}



In [356]:
parsel = Selector(res_default.text)

옵션 카테고리 수집

In [466]:
# get order category
order_names = parsel.xpath('/html/body/div[3]/div[2]/div/div[1]/div[1]/div[2]/ul/li[1]/div/div/a/text()').getall()
order_ids  = parsel.xpath('/html/body/div[3]/div[2]/div/div[1]/div[1]/div[2]/ul/li[1]/div/div/a/@onclick').getall()
order_list = []

for idx in range(len(order_names)): 
    tmp = {}
    tmp['order_name'] = order_names[idx]
    tmp['order_id']= order_ids[idx][24]
    order_list.append(tmp)

order_list

[{'order_name': '관련도순', 'order_id': '0'},
 {'order_name': '최신순', 'order_id': '1'},
 {'order_name': '오래된순', 'order_id': '2'}]

2. 스크래핑 

In [852]:
# get corresponding parameters 
def option_params(params_initial, order, time, date = ""): 
    params_tmp = {}
    params_tmp = params_initial.copy() 
    #order
    for i in order_list: 
        if i['order_name'] == order: 
            params_tmp['sort'] = i['order_id']
    #time
    params_tmp['pd'] = time_list[str(time)]
    if time == "직접입력": 
        params_tmp['ds'] = date['ds']
        params_tmp['de'] = date['de']
    
    return params_tmp      
        

In [639]:
def get_news_list(url_search, params_news, page_number=1):
    params_tmp = {}
    params_tmp = params_news.copy() 
    list_url = []
    news_list = []
    if page_number > 1:
        for i in range(1, page_number+1): 
            if i != 1:
                params_tmp['sm'] = 'tab_pge'
                params_tmp['start'] = str(i-1)+'1'
                
            res = req.get(url_search, headers = headers_default, params=params_tmp)
            soup = bs(res.text, 'html.parser')
            try:
                tmp = soup.find('ul', {'class': 'list_news'}).find_all('li')
            except: 
                print(f"Page reaches the end, page {i-1} is the end")
                break
            else:
                news_list.extend(tmp)
    else:
        res = req.get(url_search, headers = headers_default, params=params_tmp)
        soup = bs(res.text, 'html.parser')
        news_list = soup.find('ul', {'class': 'list_news'}).find_all('li')
     
    
    for news in news_list: 
        try: 
            tmp2 = news.find('div', {'class': 'info_group'}).find_all('a')[1]['href']
        except: 
            print("No 네이버뉴스 format")
        else: 
            list_url.append(news.find('div', {'class': 'info_group'}).find_all('a')[1]['href'])

    return list_url



In [438]:
def get_ids(url_single): 
    aid = url_single[-10:]
    oid = url_single[-18:-15]
    sid = url_single[-26:-23]
    return aid, oid, sid

In [858]:
# User Inputs
query = "올림픽"
order_option = "최신순"
time_option = "직접입력"
# optional: 직접 입력 date
date = {'ds': "2021.07.06", 'de': "2021.07.12"}
page_number = 8 
data_category = ['기사제목']

In [855]:
# scraping starts
params_search = {}
params_search = option_params(params_default, order_option, time_option, date)
params_search['query'] = query

url_read="https://news.naver.com/main/read.nhn?mode=LSD&mid=shm"
params_read = {}
news_data = []
url_news_list = get_news_list(url_default, params_search, page_number)

for url_single in url_news_list: 
    aid, oid, sid = get_ids(url_single)
    params_read['sid1'] = sid
    params_read['oid'] = oid
    params_read['aid'] = aid
    res_temp = req.get(url_single, headers = headers_default)
    url_temp = res_temp.url
    if url_temp[8] == 's':
        print("Sports Category")
        params_read.pop('sid1')
        page_info = get_info_sports(params_read)
    elif url_temp[8] == 'e':
        print('Entertain Category')
        params_read.rpop('sid1')
        page_info = get_info_entertain(params_read)
    else:
        print('General Category')
        page_info = get_info(url_read, params_read)
    news_data.append(page_info)

news_data

No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
No 네이버뉴스 format
Sports Category
호텔조식 방에서, 저녁은 편의점 도시락…도쿄올림픽 '식사 전쟁'
General Category
'올림픽 강행' 스가 내각 지지율 33%…출범 이후 최저
General Category
"강제노동 없었다" 주장하더니...日, 올림픽 앞두고 체면 구겨
Sports Category
이강인과 만났던 아르헨티나 감독, "한국은 강팀"
General Category
日, 올림픽 앞두고 도쿄에 금주령... 선수촌 술 반입은 허용
Sports Category
NC·두산, 프로야구 중단 얻었지만 '팬 기만' 드러났다
Sports Category
[오피셜] '도쿄 올림픽도 뛴다' 페드리, 유로 영 플레이어 수상
Sports Category
코로나 여파에… 프로야구 13일부터 리그 전격 중단
Sports Category
韓MVP, 日 벽 못 넘나. 두 번째 2군행 굴욕 확정
Sports Category
선수단 잇단 확진…프로야구도 중단한다
Sports Category
일본, '韓 같은조' 온두라스에 3-1 승... 17일 스페인과 최종 평가전
General Category
도쿄 코로나 확진자 두 달 만에 월요일 기준 500명 넘어
General Category
'긴급사태' 도쿄 

[{'언론사 이름': 'MHN스포츠',
  '기사제목': "호텔조식 방에서, 저녁은 편의점 도시락…도쿄올림픽 '식사 전쟁'",
  '기사입력시각': '기사입력 2021.07.12. 오후 11:57',
  '반응수': 5,
  '기사본문': '[MHN스포츠 이규원 기자] 일본을 방문하는 도쿄올림픽 관계자들은 신종 코로나바이러스 감염증(코로나19) 검사를 6번 받아야한다.\n코로나19 검사에서 음성이 나와도 호텔에서 3일간 엄격한 격리생활을 해야한다.\xa0\n격리 생활에서 가장 힘든 점은 끼니 해결이다.\n조식은 호텔에서 뷔페식으로 제공하는데, 음식을 싸서 각자 방으로 돌아가 식사한다.\n문제는 스스로 해결해야 하는 중식과 저녁이다.\n조직위는 편의점에서 도시락을 구매해 식사하는 것을 권유한다.\n단, 도시락 구매를 위해 허용하는 외출 시간은 단 15분이다.\n15분을 넘길 경우 조직위로부터 경고를 받게 되고, 심할 경우 강제 추방당할 수 있다.\n편의점이 호텔에서 멀리 떨어져 있거나 구매하는데 시간이 지체되면 문제가 생긴다.\n12일 몇몇 한국 취재진은 뛰어서 도시락을 구매해오기도 했다. 밥 한 끼 먹기가 여간 힘든 게 아니다.\n격리 생활의 질은 정보 통신 기기 사용 능력에 따라 갈리기도 한다.\n스마트폰 활용 능력이 뛰어난 한 취재진은 현지 애플리케이션을 다운받아 그럴싸한 배달 음식을 시켜 먹었다.\n그나마 편의점 도시락이 익숙하고 비교적 배달 애플리케이션 사용이 능숙한 한국 취재진은 격리 생활이 나은 편이다.\n반면 일본 음식이 입맛에 맞지 않는 서양 관계자들은 곤혹스러운 시간을 보낼 수 있다.\n특히 채식주의자나 할랄 음식을 먹어야 하는 무슬림 관계자들의 여건은 열악하다.\n편의점에선 이들을 위한 음식을 쉽게 찾을 수 없다. 배달 음식도 마찬가지다.\n다행히 지금까지 불상사가 발생했다는 소식은 들리지 않았다.\n한편, 도쿄올림픽 조직위원회는 해외에서 유입하는 코로나19를 원천 봉쇄한다며 엄격한 방역 수칙을 입국자들에게 요구한다.\n한국 취재진도 이를 따

In [845]:
final_data = pd.DataFrame(news_data)

In [859]:
final_data[data_category]

,기사제목
0,히딩크가 극찬한 골키퍼 차기석 앗아간 ‘만성신부전’ 의심해야 할 증상은?
1,"故 차기석 사망 소식에 선수협, “애통하고 슬프다”"
2,"영국 매체, 한국축구 도쿄올림픽 4강 진출 예상"
3,"히딩크가 인정한 골키퍼 차기석, 35세에 하늘로…"
4,"히딩크도 주목했던 GK 유망주 차기석, 투병 끝에 별세"
5,"히딩크도 극찬한 골키퍼 차기석, 투병 끝 35세에 하늘로"
6,"히딩크 감독이 주목했던 차기석, 투병 끝에 13일 별세"
7,"‘전 국가대표 골키퍼’ 차기석, 투병 끝에 향년 35세로 별세"
8,"[오피셜] 구자철, 알 가라파와 작별...타 구단 입단 타진"
9,"‘알 가라파와 결별’ 구자철, 향후 행선지 물색... 국내보단 해외에 무게"


시나리오 1에도 쓰였던 함수들:

In [729]:
def get_info(url_read, params_read): 
    res = req.get(url_read, headers = headers_default, params=params_read)
    res_url = url_read
    soup = bs(res.text, 'html.parser')
    selector = Selector(res.text)
    tmp = {}
    tmp['언론사 이름'] = soup.find('div', {'class': 'article_header'}).find('div', {'class': 'press_logo'}).find('img')['alt']
    tmp['기사제목'] = soup.find('h3').text.strip()
    print(tmp['기사제목'])
    tmp['기사입력시각'] = soup.find('div', {'class': 'article_info'}).find('div').find('span').text.strip()
    tmp['반응수'] = get_reaction(res, params_read, res_url)
    tmp['댓글수'], tmp['댓글모음'] =  get_comment(res, params_read, res_url)
    tmp['기사본문'] = '\n'.join(selector.xpath('//*[@id="articleBodyContents"]/text()').getall()).strip()
    if soup.find('div', {'class': 'journalistcard_summary_name'}) != None:
        tmp['기자명'] = soup.find('div', {'class': 'journalistcard_summary_name'}).text 
    else: 
        tmp['기자명'] = ""
    if soup.find('div', {'class': 'journalistcard_email'}) != None:
        tmp['기자 이메일'] = soup.find('div', {'class': 'journalistcard_email'}).text
    else: 
        tmp['기자 이메일'] = ""
    return tmp 

In [730]:
def get_info_sports(params_sports, url_sports = "https://sports.news.naver.com/news"): 
    res = req.get(url_sports, headers = headers_default, params = params_sports)
    res_url = url_sports
    soup = bs(res.text, 'html.parser')
    selector = Selector(res.text)
    tmp = {}
    tmp['언론사 이름'] = soup.find('span', {'id': 'pressLogo'}).find('img')['alt']
    tmp['기사제목'] = soup.find('h4').text.strip()
    print(tmp['기사제목'])
    tmp['기사입력시각'] = soup.find('div', {'class': 'news_headline'}).find('div', {'class': 'info'}).find('span').text.strip()
    tmp['반응수'] = get_reaction_sports(res, params_sports, res_url)
    tmp['기사본문'] = '\n'.join(selector.xpath('//*[@id="newsEndContents"]/text()').getall()).strip()
    if soup.find('div', {'class': 'reporter_profile'}) != None:
        tmp['기자명'] = soup.find('div', {'class': 'reporter_profile'}).find('div', {'class': 'name'}).text 
    else: 
        tmp['기자명'] = ""
    if soup.find('div', {'class': 'reporter'}) != None:
        tmp['기자 이메일'] = soup.find('div', {'class': 'reporter'}).find('p').text
    else: 
        tmp['기자 이메일'] = ""
    return tmp 

In [731]:
def get_info_entertain(params_entertain, url_entertain='https://entertain.naver.com/read' ):
    res = req.get(url_entertain, headers = headers_default, params = params_entertain)
    res_url = url_entertain
    soup = bs(res.text, 'html.parser')
    selector = Selector(res.text)
    tmp = {}
    tmp['언론사 이름'] = soup.find('div', {'class': 'press_logo'}).find('img')['alt']
    tmp['기사제목'] = soup.find('h2', {'class': 'end_tit'}).text.strip()
    print(tmp['기사제목'])
    tmp['기사입력시각'] = soup.find('div', {'class': 'article_info'}).find('em').text.strip()
    tmp['반응수'] = get_reaction_entertain(res, params_entertain, res_url)
    tmp['기사본문'] = '\n'.join(selector.xpath('//*[@id="articeBody"]/text()').getall()).strip()
    if soup.find('div', {'class': 'journalistcard_summary_name'}) != None:
        tmp['기자명'] = soup.find('div', {'class': 'journalistcard_summary_name'}).text 
    else: 
        tmp['기자명'] = ""
    if soup.find('div', {'class': 'journalistcard_email'}) != None:
        tmp['기자 이메일'] = soup.find('div', {'class': 'journalistcard_email'}).text
    else: 
        tmp['기자 이메일'] = ""
    return tmp 

In [723]:
url_entertain='https://entertain.naver.com/read'
params_entertain = {}
params_entertain['oid'] = '117'
params_entertain['aid'] =  '0003514316'
res = req.get(url_entertain, headers = headers_default, params = params_entertain)
res_url = url_entertain
soup = bs(res.text, 'html.parser')
selector = Selector(res.text)
a = soup.find('div', {'class': 'press_logo'}).find('img')['alt']
b = soup.find('h2', {'class': 'end_tit'}).text.strip()
c = soup.find('div', {'class': 'article_info'}).find('em').text.strip()
d = '\n'.join(selector.xpath('//*[@id="articeBody"]/text()').getall()).strip()
k = soup.find('div', {'class': 'journalistcard_summary_name'}).text.strip() 
f = soup.find('div', {'class': 'journalistcard_email'}).text.strip()
g = get_reaction_entertain(res, params_entertain, res_url)
g

45

In [658]:
headers_json = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
#     'cookie': 'NNB=WOBIUKCSTPQV6; ASID=daeb16c20000017689de56c700000061; NRTK=ag#all_gr#0_ma#-2_si#-2_en#-2_sp#-2; NDARK=Y; _ga_4BKHBFKFK0=GS1.1.1611817403.1.1.1611817598.60; _ga_1BVHGNLQKG=GS1.1.1621914872.1.1.1621915119.0; NV_WETR_LAST_ACCESS_RGN_M="MDkxNDAxMDQ="; NV_WETR_LOCATION_RGN_M="MDkxNDAxMDQ="; nid_inf=504467378; NID_AUT=ZX/UlfQLyx50eE7GIfZOz1K/JfKNlsahJD7RRpvfbPWvRg8Itk4ZEqtCiCM0N/0g; NID_JKL=Gly790Xk30zqIlsGAOYIY1U4974HqqU+7cxTnbPXgWI=; _gid=GA1.2.2064746431.1625562488; _ga=GA1.2.260803929.1608800109; nx_ssl=2; _ga_7VKFYR6RV1=GS1.1.1625569413.100.1.1625569413.60; NID_SES=AAABkzKeM8IWBdJg8QQagX/6+v4olPW0qQr1pGjTJmA+Zls8nxwULiP+jgZMx3+eQVPCs9isrHN4F090Aaw0BhkWZxgLLZ9r1CP7JTLus4L+yHgKrH2EGWID7IVz4JWAPI+aeGCTjl8dnNiS7l67tdDthxYtGyGdfAN35zW2S+2WjCOIRplfeVITmNTFQK8qt7XnDNjKq8t4GsDmEOUnGVROqZo0qfph3BO41QRc8XXDL5Spc5xMqfWLLzN7+f3/AP3bZkX1sEloOGkfeywD9SIo0U/dP/a1e27bSGaAbAHyI8iVOvFjgJTR4+YYOZiXB9qS8QECmOd4roBwHEvBwsm3mEAc1W9NbjQtbNKWKrbZS7wWJa/34NFBEH5MFMgF5cTQ3n1UisowGXM9Aaqxb5HfbH84fCqEdvQMBmKFbosGmJHmu5bwj11VOTEkzVcfqjPqfXL6PMVGPCMHd1B9fXkFIhcQz8Q3oFMbuHkKB2wYEoUmbDSwwfAUcieZ5l74Y6q7CwvpTWAqjwnxvyltDHiaIGmoyC/6lZ2NgoPxX2c8uZPZ'
#     'referer': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=088&aid=0000711141',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'sec-ch-ua-mobile': '?0',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
}

In [732]:
def get_reaction(res, params_read, res_url, url2 = 'https://news.like.naver.com/v1/search/contents', headers = headers_json):

    sid = params_read['sid1']
    oid = params_read['oid']
    aid = params_read['aid']
    headers['referer'] = res_url+f'&sid1={sid}&oid={oid}&aid={aid}'
    soup = bs(res.text, 'html.parser')
    params = {'suppress_response_codes': 'true', 'isDuplication': 'false'}
    a =soup.find('a', {'class': 'journalistcard_summary_link'})
    if a != None:
        journal_id = a['href'][-5:]
        params['q'] = f'NEWS[ne_{oid}_{aid}]|NEWS_SUMMARY[{oid}_{aid}]|JOURNALIST[{journal_id}(period)]|NEWS_MAIN[ne_{oid}_{aid}]'
    else: 
        params['q'] = f'NEWS[ne_{oid}_{aid}]|NEWS_SUMMARY[{oid}_{aid}]|NEWS_MAIN[ne_{oid}_{aid}]'
#     params['_'] = '1625559039649'
#     params['callback'] = 'jQuery112406548342014790987_1625559039648'
    res2 = req.get(url2, headers=headers, params = params)
#     js = json.loads(res2.text.split('1625559039648(')[-1].split(');')[0])
    js = res2.json()
    total = 0
    for react in js['contents'][0]['reactions']:
        total += react['count']
    return total

In [733]:
def get_reaction_sports(res, params_read, res_url, url2 ='https://sports.like.naver.com/v1/search/contents', headers=headers_json):
    oid = params_read['oid']
    aid = params_read['aid']
    headers['referer'] = res_url+f'?oid={oid}&aid={aid}'
    soup = bs(res.text, 'html.parser')
    params = {'suppress_response_codes': 'true', 'isDuplication': 'false'}
    a = soup.find('div', {'class': 'profile_info'})
    if a != None: 
        b = a.find('a', {'class': 'link_press'})
        journal_id = b['href'][-5:]
        params['q'] = f'SPORTS[ne_{oid}_{aid}]|JOURNALIST[{journal_id}(period)]|SPORTS_MAIN[ne_{oid}_{aid}]'
    else: 
        params['q'] = f'SPORTS[ne_{oid}_{aid}]|SPORTS_MAIN[ne_{oid}_{aid}]'
    res2 = req.get(url2, headers=headers, params = params)
    js = res2.json()
    total = 0
    for react in js['contents'][0]['reactions']:
        total += react['count']
    return total

In [839]:
def get_reaction_entertain(res, params_read, res_url, url3="https://news.like.naver.com/v1/search/contents", headers=headers_json): 
    oid = params_read['oid']
    aid = params_read['aid']
    headers['referer'] = res_url+f'?oid={oid}&aid={aid}'
    soup = bs(res.text, 'html.parser')
    params = {'suppress_response_codes': 'true', 'isDuplication': 'false'}
    a =soup.find('a', {'class': 'journalistcard_summary_link'})
    if a != None:
        journal_id = a['href'][-5:]
        params['q'] = f'ENTERTAIN[ne_{oid}_{aid}]|JOURNALIST[{journal_id}(period)]|ENTERTAIN_MAIN[ne_{oid}_{aid}]'
    else: 
        params['q'] = f'ENTERTAIN[ne_{oid}_{aid}]|ENTERTAIN_MAIN[ne_{oid}_{aid}]'
    res2 = req.get(url3, headers=headers, params = params)
    js = res2.json()
    total = 0
    for react in js['contents'][0]['reactions']:
        total += react['count']
    return total

In [836]:
params_sbs = {'suppress_response_codes': 'true', 'isDuplication': 'false'}
url3="https://news.like.naver.com/v1/search/contents"
oid= '382'
aid= '0000920879'
head = headers_json.copy()
head['referer'] = 'https://entertain.naver.com/read?oid=382&aid=0000920879'
params_sbs['q'] = f'ENTERTAIN[ne_{oid}_{aid}]|JOURNALIST[75232(period)]|ENTERTAIN_MAIN[ne_{oid}_{aid}]'
res2 = req.get(url3, headers=head, params = params_sbs)
js = res2.json()
total = 0
for react in js['contents'][0]['reactions']:
    total += react['count']
print(total)

6


In [735]:
def get_comment(res, params_read, res_url, comment_page= 1, url3 = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json", headers = headers_json): 
    oid = params_read['oid']
    aid = params_read['aid']
    sid = params_read['sid1']
    headers['referer'] = res_url+f'&sid1={sid}&oid={oid}&aid={aid}'
    params = {
    'ticket': 'news',
#     'templateId': 'view_politics_m3',
    'pool': 'cbox5',
#     '_callback': 'jQuery11240639671411049658_1625630484104',
    'lang': 'ko',
    'country': 'KR',
    'objectId': f'news{oid},{aid}',
    'categoryId': '',
    'pageSize': '10',
    'indexSize': '10',
    'groupId': '',
    'listType': 'OBJECT',
    'pageType': 'more',
    'page': '1',
    'initialize': 'true',
    'userType': '',
    'useAltSort': 'true',
    'replyPageSize': '20',
    'sort': 'FAVORITE',
    'includeAllStatus': 'true',
#     '_': '1625630484106'
}
    res2 = req.get(url3, headers=headers, params=params)
    js = json.loads(res2.text.split('_callback(')[-1].split(');')[0])
#     number_per_page = len(js['result']['commentList'])
#     print(f"한페이지당 로드되는 댓글수: {number_per_page}")
    count = js['result']['count']['comment']
    comment_list = []
    for i in range(1, count//10+2): 
        if i != 1:
            params['page']= i
            res2 = req.get(url3, headers=headers, params=params)
            js = json.loads(res2.text.split('_callback(')[-1].split(');')[0]) 
        for comment in js['result']['commentList']:
            comment_list.append(comment['contents'])
    
    return count, comment_list 

In [ ]:
## 불필요한 코드 부분: type and publisher option
# get type category
type_names = parsel.xpath('/html/body/div[3]/div[2]/div/div[1]/div[1]/div[2]/ul/li[3]/div/div/a/text()').getall()
type_ids = parsel.xpath('/html/body/div[3]/div[2]/div/div[1]/div[1]/div[2]/ul/li[3]/div/div/a/@onclick').getall()
type_list = []

for idx in range(len(type_names)): 
    tmp = {}
    tmp['type_name'] = type_names[idx]
    tmp['type_id']= type_ids[idx][28]
    type_list.append(tmp)
    
type_list 
# get publisher categories
script = parsel.xpath('/html/body/div[3]/div[2]/div/div[1]/script[4]/text()').getall()
script = script[0]
# remove unneccessary parts at beginning and the end
script = script[script.find('listData'):].strip()
script = script[13:]
script = script.split(');')[0]

# split the lists into '언론사 분류순' and '가나다순'
split_script_list = script.split('] }, ')
# 언론사 분류순 카테고리
media_category = split_script_list[0]

result = []
for i in media_category.split(' ] } , '):
    try:
        result.append(json.loads(i + ']}'))
    except:
        result.append(json.loads(i))

#가나다순 카테고리
media_initial = split_script_list[1]
media_initial = media_initial[media_initial.find('listData'):].strip()
media_initial = media_initial[13:]
media_initial = media_initial[:-8]
result2 = []
for i in media_initial.split(', ] } , '):
    try:
        result2.append(json.loads(i + ']}'))
    except:
        result2.append(json.loads(i[:-5] + i[-4:]))